# Embedding Analysis

In [5]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from pipeline import Pipeline

## Helper Functions

In [7]:
def average_precision_at_k(retrieved_items, true_name):
    """
    Calculate the average precision at K for a single query.
    
    :param retrieved_items: List of items retrieved by the model (sorted by rank).
    :param true_name: The true name of the probe image.
    :return: Average precision at K.
    """
    num_relevant = 0
    cumulative_precision = 0.0

    for rank, item in enumerate(retrieved_items, start=1):
        if item['name'] == true_name:
            num_relevant += 1
            # Precision at this rank
            precision_at_rank = num_relevant / rank
            cumulative_precision += precision_at_rank

    # Return average precision, normalized by the number of relevant items found
    if num_relevant == 0:
        return 0.0
    else:
        return cumulative_precision / num_relevant

def mean_average_precision(true_name_list, retrieved_items_list):
    """
    Calculate the Mean Average Precision (MAP) over all probe queries.
    
    :param true_name_list: List of true names (one for each query/probe image).
    :param retrieved_items_list: List of retrieved items (list of lists for each query).
    :return: Mean Average Precision (MAP) score.
    """
    total_ap = 0.0
    num_queries = len(true_name_list)

    # Loop over all queries and calculate the average precision for each
    for true_name, retrieved_items in zip(true_name_list, retrieved_items_list):
        ap = average_precision_at_k(retrieved_items, true_name)
        total_ap += ap

    # Return the mean of the average precision scores
    return total_ap / num_queries


## Evaluation

In [8]:
def evaluate(pipeline, probe_directory, gallery_faiss_path, gallery_metadata_path, k=5, metric='euclidean'):
    """
    Evaluate the model by calculating Mean Average Precision (MAP) using all probe images.
    
    :param pipeline: The initialized Pipeline object (with the desired model).
    :param probe_directory: Directory containing probe images.
    :param gallery_faiss_path: Path to the precomputed FAISS index for the gallery.
    :param gallery_metadata_path: Path to the metadata associated with the FAISS index.
    :param k: Number of nearest neighbors to retrieve.
    :param metric: Distance metric to use in FAISS search ('euclidean', 'minkowski', etc.).
    :return: Mean Average Precision (MAP) for all probe images.
    """
    true_name_list = []
    retrieved_items_list = []
    
    for root, _, files in os.walk(probe_directory):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                probe_image_path = os.path.join(root, file)
                
                # Extract the true name from the folder name (assumed to be the name of the person)
                true_name = os.path.basename(root)
                true_name_list.append(true_name)
                
                # Perform a search with the pipeline and store the retrieved items
                retrieved_items = pipeline.search_gallery(probe_image_path, k=k, metric=metric, faiss_path=gallery_faiss_path, metadata_path=gallery_metadata_path)
                retrieved_items_list.append(retrieved_items)
    
    # Calculate Mean Average Precision (MAP)
    map_score = mean_average_precision(true_name_list, retrieved_items_list)
    
    return map_score

In [9]:
# Step 1: Initialize the 'vggface2' pipeline and precompute gallery embeddings
pipeline_vggface2 = Pipeline(pretrained='vggface2', index_type='brute_force')

# # Precompute and save embeddings for 'vggface2'
# gallery_directory = "../storage/multi_image_gallery"
# pipeline_vggface2._Pipeline__precompute(gallery_directory)
# pipeline_vggface2._Pipeline__save_embeddings(faiss_path="../storage/catalog/vggface2.index", 
#                                              metadata_path="../storage/catalog/vggface2_metadata.pkl")

# Step 2: Initialize the 'casia-webface' pipeline and precompute gallery embeddings
pipeline_casia_webface = Pipeline(pretrained='casia-webface', index_type='brute_force')

# # Precompute and save embeddings for 'casia-webface'
# pipeline_casia_webface._Pipeline__precompute(gallery_directory)
# pipeline_casia_webface._Pipeline__save_embeddings(faiss_path="../storage/catalog/casia_webface.index", 
#                                                   metadata_path="../storage/catalog/casia_webface_metadata.pkl")


## Model General Performance

From the results, it's clear that the vggface2 model significantly outperforms casia-webface across all evaluations. For all distance metrics—euclidean, dot product, cosine, and minkowski—vggface2 achieved a consistent Mean Average Precision (MAP) of about 0.54, whereas casia-webface struggled, with a MAP consistently around 0.12. This suggests that vggface2 is much better suited for this particular task of face recognition.

When analyzing the performance across different values of k (the number of nearest neighbors), we see a similar trend. For vggface2, the MAP peaks at k=5 with a score of 0.55, then slightly declines as k increases. Meanwhile, casia-webface shows much less variance across different k values, with its MAP remaining relatively low, peaking at k=15 with a score of 0.119. This consistent underperformance suggests that casia-webface struggles to retrieve relevant results effectively, even when given more neighbors to work with.

Interestingly, changing the distance metric or the k value didn't significantly impact the results for either model. The vggface2 model maintained high performance across all metrics and k values, while casia-webface continued to perform poorly. This suggests that the vggface2 model's ability to retrieve and rank relevant faces is inherently more robust, independent of these factors.

Overall, these results clearly indicate that vggface2 provides much stronger performance for the face recognition task at hand, regardless of the distance metric or k value. Since accuracy and precision are critical for this application, vggface2 would be the preferred model.

In [17]:
# Define the distance metrics to test
distance_metrics = ['euclidean', 'dot_product', 'cosine', 'minkowski']

# Evaluate the 'vggface2' model using all probe images and multiple distance metrics
probe_directory = "../simclr_resources/probe"
gallery_faiss_path_vggface2 = "../storage/catalog/vggface2.index"
gallery_metadata_path_vggface2 = "../storage/catalog/vggface2_metadata.pkl"

print("Evaluating vggface2 model on all probe images...")

# Initialize the 'vggface2' pipeline
pipeline_vggface2 = Pipeline(pretrained='vggface2', index_type='IVF')

# Loop through each distance metric and evaluate
for metric in distance_metrics:
    print(f"Evaluating vggface2 with {metric} distance...")
    map_vggface2 = evaluate(pipeline_vggface2, probe_directory, gallery_faiss_path_vggface2, gallery_metadata_path_vggface2, k=10, metric=metric)
    print(f"Mean Average Precision (MAP) for vggface2 using {metric}: {map_vggface2}")

# Evaluate the 'casia-webface' model using all probe images and multiple distance metrics
gallery_faiss_path_casia_webface = "../storage/catalog/casia_webface.index"
gallery_metadata_path_casia_webface = "../storage/catalog/casia_webface_metadata.pkl"

print("Evaluating casia-webface model on all probe images...")

# Initialize the 'casia-webface' pipeline
pipeline_casia_webface = Pipeline(pretrained='casia-webface', index_type='IVF')

# Loop through each distance metric and evaluate
for metric in distance_metrics:
    print(f"Evaluating casia-webface with {metric} distance...")
    map_casia_webface = evaluate(pipeline_casia_webface, probe_directory, gallery_faiss_path_casia_webface, gallery_metadata_path_casia_webface, k=10, metric=metric)
    print(f"Mean Average Precision (MAP) for casia-webface using {metric}: {map_casia_webface}")


Evaluating vggface2 model on all probe images...
Evaluating vggface2 with euclidean distance...
Mean Average Precision (MAP) for vggface2 using euclidean: 0.5445760627407453
Evaluating vggface2 with dot_product distance...
Mean Average Precision (MAP) for vggface2 using dot_product: 0.5445760627407453
Evaluating vggface2 with cosine distance...
Mean Average Precision (MAP) for vggface2 using cosine: 0.5445760627407453
Evaluating vggface2 with minkowski distance...
Mean Average Precision (MAP) for vggface2 using minkowski: 0.5445760627407453
Evaluating casia-webface model on all probe images...
Evaluating casia-webface with euclidean distance...
Mean Average Precision (MAP) for casia-webface using euclidean: 0.1187389770723104
Evaluating casia-webface with dot_product distance...
Mean Average Precision (MAP) for casia-webface using dot_product: 0.1187389770723104
Evaluating casia-webface with cosine distance...
Mean Average Precision (MAP) for casia-webface using cosine: 0.1187389770723

In [18]:
# Define the distance metrics to test
k_values = [2, 5, 10, 15, 20]
metric = 'euclidean'

print("Evaluating vggface2 model on all probe images...")

# Loop through each distance metric and evaluate
for k in k_values:
    print(f"Evaluating vggface2 with a k value of {k}...")
    map_vggface2 = evaluate(pipeline_vggface2, probe_directory, gallery_faiss_path_vggface2, gallery_metadata_path_vggface2, k=k, metric=metric)
    print(f"Mean Average Precision (MAP) for vggface2 using k={k}: {map_vggface2}")

print("Evaluating casia-webface model on all probe images...")

# Loop through each distance metric and evaluate
for k in k_values:
    print(f"Evaluating casia-webface with a k value of {k}...")
    map_casia_webface = evaluate(pipeline_casia_webface, probe_directory, gallery_faiss_path_casia_webface, gallery_metadata_path_casia_webface, k=k, metric=metric)
    print(f"Mean Average Precision (MAP) for casia-webface using k={k}: {map_casia_webface}")


Evaluating vggface2 model on all probe images...
Evaluating vggface2 with a k value of 2...
Mean Average Precision (MAP) for vggface2 using k=2: 0.5395395395395396
Evaluating vggface2 with a k value of 5...
Mean Average Precision (MAP) for vggface2 using k=5: 0.5537732176621064
Evaluating vggface2 with a k value of 10...
Mean Average Precision (MAP) for vggface2 using k=10: 0.5445760627407453
Evaluating vggface2 with a k value of 15...
Mean Average Precision (MAP) for vggface2 using k=15: 0.5402126574591302
Evaluating vggface2 with a k value of 20...
Mean Average Precision (MAP) for vggface2 using k=20: 0.5344032945343152
Evaluating casia-webface model on all probe images...
Evaluating casia-webface with a k value of 2...
Mean Average Precision (MAP) for casia-webface using k=2: 0.1031031031031031
Evaluating casia-webface with a k value of 5...
Mean Average Precision (MAP) for casia-webface using k=5: 0.11490101212323434
Evaluating casia-webface with a k value of 10...
Mean Average Pre

## Transormation Analysis

After evaluating both models on various image transformations, it’s clear that the models respond differently to changes in brightness, contrast, and blur, compared to their performance on the original probe images.

### vggface2 Performance:
- **Low Brightness**: The MAP dropped significantly to 0.2067 compared to the baseline of 0.54. This suggests that dimly lit images negatively affect the model's ability to retrieve relevant results.
- **High Brightness**: Performance was even lower at 0.1531, indicating that overly bright images also pose a challenge for vggface2.
- **Low Contrast**: Interestingly, vggface2 performed better with low-contrast images, achieving a MAP of 0.6078, even surpassing the baseline performance.
- **High Contrast**: The MAP dropped again to 0.2258, showing that extreme contrast also impacts retrieval accuracy.
- **Blur**: The MAP of 0.4868 indicates that blur does degrade performance, but not as severely as brightness or high contrast.

### casia-webface Performance:
- **Low Brightness**: The MAP for casia-webface dropped drastically to 0.0267, showing the model is highly sensitive to low-light conditions.
- **High Brightness**: Performance slightly improved to 0.0620, but still far below the baseline.
- **Low Contrast**: Casia-webface struggled with low-contrast images, with a MAP of 0.0917, which is better than the brightness transformations but still poor.
- **High Contrast**: The MAP was 0.0447, indicating that both extremes in contrast negatively affect casia-webface.
- **Blur**: Like vggface2, casia-webface saw a drop in performance, achieving a MAP of 0.0878 on blurred images.

### Comparison to Baseline Performance
Compared to the baseline results, vggface2 continues to outperform casia-webface across all transformations, but both models show a noticeable decline in performance under certain conditions. For vggface2, low contrast actually improved performance, while brightness transformations (both low and high) significantly harmed its accuracy. Casia-webface struggled with all transformations, performing especially poorly on brightness variations and blur, reflecting its overall weaker performance in face retrieval tasks.

In conclusion, vggface2 remains the better option, but its sensitivity to brightness and blur suggests areas for improvement. For applications dealing with varied lighting or blurry images, additional preprocessing or training might be needed to improve robustness. Casia-webface, on the other hand, consistently underperforms, making it less suitable for this face recognition task.


In [4]:
from PIL import Image, ImageEnhance, ImageFilter

def create_transformed_images(probe_directory, output_directory, transformation_type, factor):
    """
    Apply a transformation to all images in the probe directory and save them in a new output directory.
    
    :param probe_directory: The directory containing the original probe images.
    :param output_directory: The directory where the transformed images will be saved.
    :param transformation_type: The type of transformation ('brightness', 'contrast', 'blur').
    :param factor: The intensity of the transformation.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for root, _, files in os.walk(probe_directory):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                image_path = os.path.join(root, file)
                image = Image.open(image_path)
                
                # Apply the transformation
                if transformation_type == 'brightness':
                    enhancer = ImageEnhance.Brightness(image)
                    transformed_image = enhancer.enhance(factor)
                elif transformation_type == 'contrast':
                    enhancer = ImageEnhance.Contrast(image)
                    transformed_image = enhancer.enhance(factor)
                elif transformation_type == 'blur':
                    transformed_image = image.filter(ImageFilter.GaussianBlur(factor))
                else:
                    raise ValueError("Unknown transformation type")
                
                # Create the output directory 
                relative_path = os.path.relpath(root, probe_directory)  # Get relative path within the probe folder
                new_output_dir = os.path.join(output_directory, relative_path)  # Create same folder structure
                if not os.path.exists(new_output_dir):
                    os.makedirs(new_output_dir)
                
                # Save the transformed image in the new output directory
                new_filename = os.path.join(new_output_dir, file)
                transformed_image.save(new_filename)

def apply_transformations_to_probe_images(probe_directory, output_base_directory):
    """
    Apply different transformations to the probe images and save them in new directories.
    
    :param probe_directory: The directory containing the original probe images.
    :param output_base_directory: The base directory where transformed images will be saved.
    """
    transformations = [
        {'type': 'brightness', 'name': 'low_brightness', 'factor': 0.5},
        {'type': 'brightness', 'name': 'high_brightness', 'factor': 1.5},
        {'type': 'contrast', 'name': 'low_contrast', 'factor': 0.5},
        {'type': 'contrast', 'name': 'high_contrast', 'factor': 1.5},
        {'type': 'blur', 'name': 'blur', 'factor': 2}, 
    ]
    
    for transform in transformations:
        output_directory = os.path.join(output_base_directory, f"probe_{transform['name']}")
        create_transformed_images(probe_directory, output_directory, transform['type'], transform['factor'])
        print(f"Created transformed images in {output_directory}")

# Define the original probe directory and apply transformations
probe_directory = "../simclr_resources/probe"
output_base_directory = "../simclr_resources"

apply_transformations_to_probe_images(probe_directory, output_base_directory)


Created transformed images in ../simclr_resources/probe_low_brightness
Created transformed images in ../simclr_resources/probe_high_brightness
Created transformed images in ../simclr_resources/probe_low_contrast
Created transformed images in ../simclr_resources/probe_high_contrast
Created transformed images in ../simclr_resources/probe_blur


In [13]:
# Transformed probe directories
transformed_probes = ['probe_low_brightness', 'probe_high_brightness', 'probe_low_contrast', 'probe_high_contrast', 'probe_blur']

# Metric and k value
k = 10
metric = 'euclidean'

# Evaluate the 'vggface2' model on each transformed probe directory
print("Evaluating 'vggface2' model on transformed probe images...")

for probe_folder in transformed_probes:
    probe_directory = f"../simclr_resources/{probe_folder}"
    print(f"Evaluating 'vggface2' on {probe_folder}...")
    map_vggface2 = evaluate(pipeline_vggface2, probe_directory, gallery_faiss_path_vggface2, gallery_metadata_path_vggface2, k=k, metric=metric)
    print(f"Mean Average Precision (MAP) for 'vggface2' using {probe_folder}: {map_vggface2}")



Evaluating 'vggface2' model on transformed probe images...
Evaluating 'vggface2' on probe_low_brightness...
Mean Average Precision (MAP) for 'vggface2' using probe_low_brightness: 0.20674013188673693
Evaluating 'vggface2' on probe_high_brightness...
Mean Average Precision (MAP) for 'vggface2' using probe_high_brightness: 0.153176232733111
Evaluating 'vggface2' on probe_low_contrast...
Mean Average Precision (MAP) for 'vggface2' using probe_low_contrast: 0.6078001839708189
Evaluating 'vggface2' on probe_high_contrast...
Mean Average Precision (MAP) for 'vggface2' using probe_high_contrast: 0.22580359459989083
Evaluating 'vggface2' on probe_blur...
Mean Average Precision (MAP) for 'vggface2' using probe_blur: 0.48680106820097996


In [10]:
# Evaluate the 'casia-webface' model on each transformed probe directory
print("\nEvaluating 'casia-webface' model on transformed probe images...")

for probe_folder in transformed_probes:
    probe_directory = f"../simclr_resources/{probe_folder}"
    print(f"Evaluating 'casia-webface' on {probe_folder}...")
    map_casia_webface = evaluate(pipeline_casia_webface, probe_directory, gallery_faiss_path_casia_webface, gallery_metadata_path_casia_webface, k=k, metric=metric)
    print(f"Mean Average Precision (MAP) for 'casia-webface' using {probe_folder}: {map_casia_webface}")



Evaluating 'casia-webface' model on transformed probe images...
Evaluating 'casia-webface' on probe_low_brightness...
Mean Average Precision (MAP) for 'casia-webface' using probe_low_brightness: 0.026674889174889176
Evaluating 'casia-webface' on probe_high_brightness...
Mean Average Precision (MAP) for 'casia-webface' using probe_high_brightness: 0.06208311486089267
Evaluating 'casia-webface' on probe_low_contrast...
Mean Average Precision (MAP) for 'casia-webface' using probe_low_contrast: 0.09177490983046545
Evaluating 'casia-webface' on probe_high_contrast...
Mean Average Precision (MAP) for 'casia-webface' using probe_high_contrast: 0.044724486391153076
Evaluating 'casia-webface' on probe_blur...
Mean Average Precision (MAP) for 'casia-webface' using probe_blur: 0.08786564342119899
